In [1]:
import joblib
import numpy as np
import joblib
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Load model
model = joblib.load('/home/ec2-user/xgboost_model.joblib')


In [3]:
# Get S3 data
s3_path = 's3://apidataforiotproject/activity_sleep_weather_data.csv'

df = pd.read_csv(s3_path)
df.tail()


,export_timestamp,id,class_5_min,score,active_calories,average_met_minutes,equivalent_walking_distance,high_activity_met_minutes,high_activity_time,inactivity_alerts,...,weather_windspeed,weather_winddir,weather_pressure,weather_cloudcover,weather_visibility,weather_uvindex,weather_sunrise,weather_sunset,Sunrise Minutes,Sunset Minutes
178,2024-12-13 07:53:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,108.0,1026.0,75.0,3.0,0.0,07:59,15:52,479.0,952.0
179,2024-12-13 08:53:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,109.0,1026.0,75.0,3.0,0.0,07:59,15:52,479.0,952.0
180,2024-12-13 09:53:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,118.0,1026.0,75.0,3.0,0.0,07:59,15:52,479.0,952.0
181,2024-12-13 10:53:28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,104.0,1026.0,50.0,8.0,0.0,07:59,15:52,479.0,952.0
182,2024-12-13 11:53:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,114.0,1026.0,75.0,10.0,0.0,07:59,15:52,479.0,952.0


In [5]:
# Getting consistent format for model
data = df.copy()

data['activity_score'] = data['score']
data.drop(columns=['score'], inplace=True)

data['temp'] = data['weather_temp']
data['visibility'] = data['weather_visibility']


In [6]:
five_minute_data = pd.DataFrame(data[['day', 'class_5_min']])

In [7]:
def safe_len(x):
    if isinstance(x, (str, list)): 
        return len(x)
    return 0 

five_minute_data['class_5_min_length'] = five_minute_data['class_5_min'].apply(safe_len)


In [8]:
data.columns

Index(['export_timestamp', 'id', 'class_5_min', 'active_calories',
       'average_met_minutes', 'equivalent_walking_distance',
       'high_activity_met_minutes', 'high_activity_time', 'inactivity_alerts',
       'low_activity_met_minutes', 'low_activity_time',
       'medium_activity_met_minutes', 'medium_activity_time',
       'meters_to_target', 'non_wear_time', 'resting_time',
       'sedentary_met_minutes', 'sedentary_time', 'steps', 'target_calories',
       'target_meters', 'total_calories', 'day', 'activity_timestamp',
       'contributors_meet_daily_targets', 'contributors_move_every_hour',
       'contributors_recovery_time', 'contributors_stay_active',
       'contributors_training_frequency', 'contributors_training_volume',
       'met_interval', 'met_items', 'met_timestamp', 'sleep_score',
       'timestamp', 'contributors_deep_sleep', 'contributors_efficiency',
       'contributors_latency', 'contributors_rem_sleep',
       'contributors_restfulness', 'contributors_timin

In [9]:
# Extra formatting
data['feelslikemax']=data['weather_feelslike']
data['feelslike']=data['weather_feelslike']


In [10]:
data.head()

,export_timestamp,id,class_5_min,active_calories,average_met_minutes,equivalent_walking_distance,high_activity_met_minutes,high_activity_time,inactivity_alerts,low_activity_met_minutes,...,weather_uvindex,weather_sunrise,weather_sunset,Sunrise Minutes,Sunset Minutes,activity_score,temp,visibility,feelslikemax,feelslike
0,2024-12-02 15:17:43,03b6854e-fbd1-4298-b0e2-5f35a46396e8,1111122111111111111111111111121111111112233311...,629.0,2.03125,10253.0,376.0,2220.0,1.0,65.0,...,0.0,07:46,15:55,466.0,955.0,84.0,10.0,10.0,8.0,8.0
1,2024-12-02 15:17:49,03b6854e-fbd1-4298-b0e2-5f35a46396e8,1111122111111111111111111111121111111112233311...,629.0,2.03125,10253.0,376.0,2220.0,1.0,65.0,...,0.0,07:46,15:55,466.0,955.0,84.0,10.0,10.0,8.0,8.0
2,2024-12-02 15:17:53,03b6854e-fbd1-4298-b0e2-5f35a46396e8,1111122111111111111111111111121111111112233311...,629.0,2.03125,10253.0,376.0,2220.0,1.0,65.0,...,0.0,07:46,15:55,466.0,955.0,84.0,10.0,10.0,8.0,8.0
3,2024-12-02 15:53:28,03b6854e-fbd1-4298-b0e2-5f35a46396e8,1111122111111111111111111111121111111112233311...,631.0,1.96875,10268.0,376.0,2220.0,2.0,66.0,...,0.0,07:46,15:55,466.0,955.0,82.0,9.0,10.0,7.0,7.0
4,2024-12-02 16:26:49,03b6854e-fbd1-4298-b0e2-5f35a46396e8,1111122111111111111111111111121111111112233311...,711.0,2.03125,11801.0,376.0,2220.0,2.0,91.0,...,0.0,07:46,15:55,466.0,955.0,87.0,9.0,10.0,7.0,7.0


In [11]:
# Minutes before 7 am function

from datetime import datetime, timedelta

def minutes_before_7am(time_str):
    time_obj = datetime.strptime(time_str, "%H:%M")
    base_time = datetime.strptime("07:00", "%H:%M") + timedelta(days=1)
    time_diff = base_time - time_obj
    return time_diff.total_seconds() / 60



In [12]:
# 7 day average sleep
data['timestamp'] = pd.to_datetime(data['export_timestamp'])
data['date'] = data['timestamp'].dt.date
daily_avg_sleep = (
    data.groupby('date')['sleep_score']
    .mean()
    .reset_index()
    .rename(columns={'sleep_score': 'daily_avg_sleep_score'})
)

daily_avg_sleep['seven_day_average_sleep'] = daily_avg_sleep['daily_avg_sleep_score'].rolling(window=7, min_periods=1).mean()

data = pd.merge(
    data,
    daily_avg_sleep[['date', 'seven_day_average_sleep']],
    on='date',
    how='left'
)



In [13]:
# Remove unneeded columns for the model
irrelevant_columns = ['export_timestamp', 'id', 'class_5_min', 'day', 'activity_timestamp',
                      'met_items', 'met_timestamp', 'timestamp', 'weather_sunrise', 'weather_sunset']
data = data.drop(columns=irrelevant_columns)


In [14]:
# Generate possible bedtimes
def generate_bedtime_range(start_time_str, end_time_str, interval_minutes):

    start_time = datetime.strptime(start_time_str, "%H:%M")
    end_time = datetime.strptime(end_time_str, "%H:%M")
    bedtimes = []
    current_time = start_time
    while current_time < datetime.strptime("23:59", "%H:%M") + timedelta(minutes=1):  # 11:59 PM
        bedtimes.append(current_time.strftime("%H:%M"))
        current_time += timedelta(minutes=interval_minutes)
    current_time = datetime.strptime("00:00", "%H:%M")
    while current_time <= end_time:
        bedtimes.append(current_time.strftime("%H:%M"))
        current_time += timedelta(minutes=interval_minutes)

    return bedtimes

In [15]:
def minutes_to_time(minutes_before_7am):

    base_time = datetime.strptime("07:00", "%H:%M")
    final_time = base_time - timedelta(minutes=minutes_before_7am)
    return final_time.strftime("%H:%M")

In [16]:
bedtime_range = generate_bedtime_range("20:00", "01:00", 15)

# Model features
model_features = [
    'Minutes Before 7am', 'contributors_meet_daily_targets', 'seven_day_average_sleep',
    'contributors_stay_active', 'resting_time', 'contributors_efficiency', 'contributors_deep_sleep',
    'feelslikemax', 'visibility', 'target_calories', 'contributors_latency', 'contributors_restfulness',
    'feelslike', 'contributors_total_sleep', 'temp'
]

# List to store optimal bedtimes and their scores
results = []

# Loop over each point in the data
for i in range(len(data)):
    # Select the ith row in the data
    point_in_dataframe = data.iloc[i].copy()

    # New dataframe for each row in original dataset (with each possible bedtime)
    temp_data = pd.DataFrame([point_in_dataframe] * len(bedtime_range), columns=model_features)
    
    # Convert bedtimes to minutes before 7am and update the column
    bedtime_in_minutes = [minutes_before_7am(bedtime) for bedtime in bedtime_range]
    temp_data['Minutes Before 7am'] = bedtime_in_minutes

    # Predict sleep scores for all bedtimes
    sleep_scores = model.predict(temp_data)

    # Optimal bedtime
    optimal_index = sleep_scores.argmax()
    optimal_bedtime_minutes = bedtime_in_minutes[optimal_index]

    optimal_bedtime = minutes_to_time(optimal_bedtime_minutes)
    results.append({'Optimal Bedtime': optimal_bedtime, 'Sleep Score': sleep_scores[optimal_index]})


results_df = pd.DataFrame(results)
results_df


,Optimal Bedtime,Sleep Score
0,22:30,80.434639
1,22:30,80.434639
2,22:30,80.434639
3,22:30,80.434639
4,22:30,81.136528
...,...,...
178,23:45,81.866745
179,22:45,82.255379
180,22:45,82.255379
181,22:45,82.201546


In [17]:
optimal_bedtime

'22:45'

In [22]:
results_df.tail()

,Optimal Bedtime,Sleep Score
178,23:45,81.866745
179,22:45,82.255379
180,22:45,82.255379
181,22:45,82.201546
182,22:45,81.804047


In [23]:
# Assuming 'optimal_bedtime' is already calculated and is in HH:MM format
#optimal_bedtime = optimal_bedtime_obj.strftime("%H:%M")  # Ensure this is in HH:MM format

# Add the optimal bedtime to each row in the DataFrame df
df['optimal_bedtime'] = results_df['Optimal Bedtime']

# The df now has an 'optimal_bedtime' column in HH:MM format


In [24]:
df['7 Day Average Sleep'] = data['seven_day_average_sleep']

In [25]:
df.to_csv('optimal_bedtime_predictions.csv', index=False)

In [26]:
!aws s3 cp optimal_bedtime_predictions.csv s3://apidataforiotproject/optimal_bedtime_predictions.csv


upload: ./optimal_bedtime_predictions.csv to s3://apidataforiotproject/optimal_bedtime_predictions.csv


Completed 1.2 MiB/1.2 MiB (11.5 MiB/s) with 1 file(s) remaining  
upload: ./optimal_bedtime_predictions.csv to s3://apidataforiotproject/optimal_bedtime_predictions.csv


In [27]:
data.head()

,active_calories,average_met_minutes,equivalent_walking_distance,high_activity_met_minutes,high_activity_time,inactivity_alerts,low_activity_met_minutes,low_activity_time,medium_activity_met_minutes,medium_activity_time,...,weather_uvindex,Sunrise Minutes,Sunset Minutes,activity_score,temp,visibility,feelslikemax,feelslike,date,seven_day_average_sleep
0,629.0,2.03125,10253.0,376.0,2220.0,1.0,65.0,4920.0,62.0,1140.0,...,0.0,466.0,955.0,84.0,10.0,10.0,8.0,8.0,2024-12-02,85.0
1,629.0,2.03125,10253.0,376.0,2220.0,1.0,65.0,4920.0,62.0,1140.0,...,0.0,466.0,955.0,84.0,10.0,10.0,8.0,8.0,2024-12-02,85.0
2,629.0,2.03125,10253.0,376.0,2220.0,1.0,65.0,4920.0,62.0,1140.0,...,0.0,466.0,955.0,84.0,10.0,10.0,8.0,8.0,2024-12-02,85.0
3,631.0,1.96875,10268.0,376.0,2220.0,2.0,66.0,5100.0,62.0,1140.0,...,0.0,466.0,955.0,82.0,9.0,10.0,7.0,7.0,2024-12-02,85.0
4,711.0,2.03125,11801.0,376.0,2220.0,2.0,91.0,6600.0,101.0,1800.0,...,0.0,466.0,955.0,87.0,9.0,10.0,7.0,7.0,2024-12-02,85.0
